#Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pytorch-lightning 
!pip install wandb

In [ ]:
!wandb login

#Imports

In [ ]:
import pandas as pd
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as LR
from tqdm.notebook import tqdm 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import seaborn as sns

#Datasets

In [ ]:
class AutoregressiveDataset(Dataset):
    '''
    dataset for an autoregressive task, the output is the input sequence shifted by 1
    '''
    def __init__(self, path:str, set_type:str, val_size:int, test_size:int, window_size:int = 7, cols_to_use = ['Open'], col_to_sort:str = 'Date',
                 scaler = None, scale_sections:int = 1, scaler_type:str = 'minmax'):
        super().__init__()
        assert (set_type in {'train', 'val', 'test'})
        assert (scaler_type in {'standard', 'minmax'})

        #read file
        self.path = path
        self.window_size = window_size

        if self.path == 'sns_flights':
            flight_data = sns.load_dataset("flights")
            npy_df = flight_data['passengers'].values.astype(float).reshape(-1,1)
        else:
            pandas_df = pd.read_csv(path).sort_values(by=col_to_sort, ascending=True)
            npy_df = np.array(pandas_df[cols_to_use]) 

        npy_df = npy_df[~np.isnan(npy_df).any(axis=1),:] #remove rows with missing values 
        assert (not np.isnan(npy_df).any())

        #split dataset
        tot_len = npy_df.shape[0]
        start = 0
        if set_type == 'test':
            start = tot_len - test_size - window_size
        elif set_type == 'val':
            start = tot_len - test_size - val_size - 2 * window_size
        
        end = tot_len
        if set_type == 'val':
            end = tot_len - test_size - window_size
        elif set_type == 'train':
            end = tot_len - test_size - val_size - 2 * window_size
        
        npy_df = npy_df[start : end]
        npy_df = self.get_whole_sequence_from_df(npy_df)
        self.unscaled_sequence = torch.tensor(npy_df)

        #normalization        
        if scaler is not None:
            self.scaler = scaler
            npy_df = scaler.transform(npy_df)
        else:
            scale_window = int((end - start) / scale_sections)
            for i in range(scale_sections):
                s = scale_window * i
                e = (end - start) if (i == scale_sections-1) else s+scale_window 
                if scaler_type == 'minmax':
                    self.scaler = MinMaxScaler(feature_range=(-1,1))    
                elif scaler_type == 'standard':
                    self.scaler = StandardScaler(with_mean=True, with_std=True)
                else:
                    self.scaler = None
                    break

                self.scaler.fit(npy_df[s:e])
                npy_df[s:e] = self.scaler.transform(npy_df[s:e])

        #transform in pytorch
        self.sequence = torch.tensor(npy_df) #.float()
        self.val_size = val_size
        self.test_size = test_size
        self.cols_to_use = cols_to_use
        self.col_to_sort = col_to_sort
        self.scale_sections = scale_sections
        self.dataset_type = 'AutoregressiveDataset'
        self.scaler_type = scaler_type

    def get_whole_sequence_from_df(self, npy_df):
        return npy_df

    def __len__(self):
        return self.sequence.shape[0] - self.window_size

    def __getitem__(self, idx):
        x = self.sequence[idx : idx + self.window_size]
        y = self.sequence[idx + 1 : idx + self.window_size + 1]
        return {'x': x, 'y': y}

    def unscaled_item(self, idx):
        x = self.unscaled_sequence[idx : idx + self.window_size]
        y = self.unscaled_sequence[idx + 1 : idx + self.window_size + 1]
        return {'x': x, 'y': y}

    def num_features(self):
        return self.sequence.shape[1]

    def get_targets(self):
        return self.sequence[self.window_size:]
    
    def get_real_prices(self):
        if self.scaler is None:
            return self.get_targets().numpy()
        return self.scaler.inverse_transform(self.get_targets().numpy())
    
    def get_predicted_prices(self, model:pl.LightningModule, teacher = False):
        if self.scaler is None:
            return self.predict(model, teacher).numpy()
        return self.scaler.inverse_transform(self.predict(model, teacher).numpy())
    
    def is_prediction_different_from_price(self):
        return self.scaler is not None

    def can_output_prices(self):
        return self.cols_to_use

    def predict(self, model:pl.LightningModule, teacher = False):
        y_pred = torch.Tensor()
        y_pred = y_pred.to(model.device)
        model.eval()
        x = self[0]['x']
        with torch.no_grad():
            for i in range(len(self)):
                if teacher:
                    x = self[i]['x']
                y_hat = model(x.unsqueeze(dim=0).to(model.device))
                y_hat = y_hat[0,-1,:].unsqueeze(dim=0)
                x = torch.cat((x, y_hat), dim=0)
                x = x[1:]
                y_pred = torch.cat((y_pred, y_hat), dim=0)
                
        return y_pred.cpu()      

    def __str__(self):
        return f'path:{self.path}, window_size:{self.window_size}, split_perc:{self.split_perc}'
    
    def log(self, config):
        config.dataset_type = self.dataset_type
        config.path = self.path
        config.window_size = self.window_size
        config.val_size = self.val_size
        config.test_size = self.test_size
        config.cols_to_use = self.cols_to_use 
        config.col_to_sort = self.col_to_sort 
        config.scale_sections = self.scale_sections
        config.scaler_type = self.scaler_type
    
    def plotPredictions(self, model, teacher, y_axis_name, plot_name, log, target_idx):
        y = self.get_targets()
        y_pred = self.predict(model, teacher)
        plotPrediction(y, y_pred, y_axis_name, plot_name, log=log, idx_target=target_idx)
        return y, y_pred
    
    def feature_name(self, i):
        return self.cols_to_use[i]

In [ ]:
class AutoregressiveDistanceDataset(AutoregressiveDataset):
    '''
    dataset[i] = sequence[i+1] - sequence[i]: we predict difference in prices not prices.
    '''

    def __init__(self, path:str, set_type:str, val_size:int, test_size:int, window_size:int = 7, cols_to_use = ['Open'], col_to_sort:str = 'Date',
                 scaler:MinMaxScaler = None, scale_sections:int = 1, scaler_type:str = 'minmax'):
        super().__init__(path, set_type, val_size, test_size, window_size, cols_to_use, col_to_sort, scaler, scale_sections, scaler_type)
        self.dataset_type = 'AutoregressiveDistanceDataset'

    def get_whole_sequence_from_df(self, npy_df):
        self.start_value = torch.tensor(npy_df[self.window_size])
        return npy_df[1:] - npy_df[:-1]
    
    def prices_from_differences(self, differences:torch.Tensor, start:torch.Tensor):
        '''
        differences has shape Len x NumFeatures
        start has shape NumFeatures (the starting values)
        '''
        ret = start.unsqueeze(-1)
        actual = start.clone().detach()
        for i in range(differences.shape[0]):
            actual += differences[i]
            ret = torch.cat((ret, actual.unsqueeze(-1)))
        return ret

    def get_real_prices(self):
        return self.prices_from_differences(torch.tensor(self.scaler.inverse_transform(self.get_targets().numpy())), self.start_value).numpy()
    
    def get_predicted_prices(self, model:pl.LightningModule, teacher = False):
        return self.prices_from_differences(torch.tensor(self.scaler.inverse_transform(self.predict(model, teacher).numpy())), self.start_value).numpy()
    
    def is_prediction_different_from_price(self):
        return True


#Losses

In [ ]:
def expanded_trend_loss4(y, y_pred):
    '''
    this is the loss described in our report, the others are for testing
    '''
    real_diff = torch.exp(y[:, 1:] - y[:, :-1])
    pred_diff = torch.exp(y_pred[:, 1:] - y_pred[:, :-1])
    tot_diff = torch.tanh(real_diff - pred_diff)
    return torch.mean(tot_diff * tot_diff)

def trend_loss(y, y_pred):
    real_diff = y[:, 1:] - y[:, :-1]
    pred_diff = y_pred[:, 1:] - y_pred[:, :-1]
    tot_diff = real_diff - pred_diff
    return torch.mean(tot_diff * tot_diff)

def expanded_trend_loss(y, y_pred):
    real_diff = torch.exp(y[:, 1:] - y[:, :-1])
    pred_diff = torch.exp(y_pred[:, 1:] - y_pred[:, :-1])
    tot_diff = real_diff - pred_diff
    return torch.mean(tot_diff * tot_diff)

def expanded_trend_loss2(y, y_pred):
    real_diff = y[:, 1:] - y[:, :-1]
    pred_diff = y_pred[:, 1:] - y_pred[:, :-1]
    tot_diff = torch.sigmoid(real_diff - pred_diff) - 0.5
    return torch.mean(tot_diff * tot_diff)

def expanded_trend_loss3(y, y_pred):
    real_diff = torch.exp(y[:, 1:] - y[:, :-1])
    pred_diff = torch.exp(y_pred[:, 1:] - y_pred[:, :-1])
    tot_diff = torch.sigmoid(real_diff - pred_diff) - 0.5
    return torch.mean(tot_diff * tot_diff)

def apply_loss(loss_name, y, y_pred, params={}):
    if loss_name == 'mse':
        return F.mse_loss(y, y_pred)
    if loss_name == 'trend':
        return trend_loss(y, y_pred)
    if loss_name == 'exp_trend':
        return expanded_trend_loss(y, y_pred)
    if loss_name == 'exp_trend2':
        return expanded_trend_loss2(y, y_pred)
    if loss_name == 'exp_trend3':
        return expanded_trend_loss3(y, y_pred)
    if loss_name == 'exp_trend4':
        return expanded_trend_loss4(y, y_pred)
    if loss_name == 'mse+trend':
        return F.mse_loss(y, y_pred) + params.get('lambda', 1.0) * trend_loss(y, y_pred)
    if loss_name == 'mse+exp_trend3':
        return F.mse_loss(y, y_pred) + params.get('lambda', 1.0) * expanded_trend_loss3(y, y_pred)
    if loss_name == 'mse+exp_trend4':
        return F.mse_loss(y, y_pred) + params.get('lambda', 1.0) * expanded_trend_loss4(y, y_pred)

#Models

In [ ]:
class GeneralModel(pl.LightningModule):
    '''class to implement some lightning methods common to all models'''
    def __init__(self, loss_name, learning_rate, use_lr_sched, lr_decay_step, lr_decay_gamma):
        super().__init__()
        self.loss_lambda = 1
        self.loss_name = loss_name
        self.initial_learning_rate = learning_rate
        self.use_lr_sched = use_lr_sched
        self.lr_decay_step = lr_decay_step
        self.lr_decay_gamma = lr_decay_gamma
        self.target_for_current_step = None

    def step(self, batch):
        x = batch['x']
        y = batch['y']
        self.target_for_current_step = y #some models might need y for the forward
        y_hat = self.forward(x)
        return apply_loss(self.loss_name, y, y_hat, {'lambda': self.loss_lambda})

    def training_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx, dataset_idx = 0):
        loss = self.step(batch)
        self.log('test_loss', loss, logger=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.initial_learning_rate)
        if self.use_lr_sched:
            lr_sched = LR.StepLR(optimizer, step_size=self.lr_decay_step, gamma=self.lr_decay_gamma)
            return {'optimizer': optimizer, 'lr_scheduler': lr_sched}
        return optimizer

    def number_of_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    #def training_epoch_end(self, training_step_outputs):
    #    pass

##MLP

In [ ]:
class MLP(GeneralModel):
    '''MLP shared across all sequence'''
    
    def __init__(self, input_size, output_size, hidden_size = 100, num_hidden_layers = 1, use_lr_sched = True, learning_rate = 1e-3, lr_decay_step = 50, lr_decay_gamma = 0.1, loss_name:str = 'mse'):
        super().__init__(loss_name, learning_rate, use_lr_sched, lr_decay_step, lr_decay_gamma)
        self.inputlayer = nn.Linear(input_size, hidden_size)
        self.hiddenlayers = nn.ModuleList([])
        for i in range(num_hidden_layers):
            self.hiddenlayers.append(nn.Linear(hidden_size, hidden_size))
        self.outputlayer = nn.Linear(hidden_size, output_size)

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size

    def forward(self, x): #used for inference
        x = self.inputlayer(x)
        for hl in self.hiddenlayers:
            x = F.relu(hl(x))
        return self.outputlayer(x)

##LSTM based

In [ ]:
class StackedLSTM(GeneralModel):
    '''standard LSTM model (the first model included in the report) '''
    
    def __init__(self, input_size, output_size, hidden_size = 100, num_layers = 1, dropout_prob = 0.0, feed_forward_depth = 1,
                 use_lr_sched = True, learning_rate = 1e-3, lr_decay_step = 50, lr_decay_gamma = 0.1, loss_name:str = 'mse', loss_lambda:float = 1.0):
        super().__init__(loss_name, learning_rate, use_lr_sched, lr_decay_step, lr_decay_gamma)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_prob, batch_first=True)
        self.fc = nn.ModuleList([])
        for i in range(feed_forward_depth - 1):
            self.fc.append(nn.Linear(hidden_size, hidden_size))
        self.fc.append(nn.Linear(hidden_size, output_size))

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.loss_lambda = loss_lambda

    def forward(self, x): #used for inference
        hiddens = (torch.zeros((self.num_layers, x.shape[0], self.hidden_size)).to(self.device),
                    torch.zeros((self.num_layers, x.shape[0], self.hidden_size)).to(self.device))
        x, hiddens = self.lstm(x, hiddens)
        for i in range(len(self.fc)-1):
            x = F.relu(self.fc[i](x))
        x = self.fc[-1](x)
        return x

In [ ]:
class EncoderDecoderLSTM(GeneralModel):
    '''LSTM with encoder/decoder paradigm (performances are not much different from standard lstm)'''
    def __init__(self, input_size, output_size, hidden_size = 100, num_layers = 1, dropout_prob = 0.0, use_lr_sched = True, learning_rate = 1e-3, lr_decay_step = 50, lr_decay_gamma = 0.1,
                 loss_name:str = 'mse', loss_lambda:float = 1.0, decoder_input = 'original_input'):
        super().__init__(loss_name, learning_rate, use_lr_sched, lr_decay_step, lr_decay_gamma)
        assert (decoder_input in [None, 'original_input', 'encoder_output'])
        self.encoder = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_prob, batch_first=True)
        dec_in = input_size
        if decoder_input == 'encoder_output':
            dec_in = hidden_size
        self.decoder = nn.LSTM(input_size=dec_in, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_prob, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.loss_lambda = loss_lambda
        self.decoder_input = decoder_input

    def forward(self, x): #used for inference
        hiddens = (torch.zeros((self.num_layers, x.shape[0], self.hidden_size)).to(self.device),
                    torch.zeros((self.num_layers, x.shape[0], self.hidden_size)).to(self.device))
        out, hiddens = self.encoder(x, hiddens) #encode the sequence in the hidden state
        dec_in = x
        if self.decoder_input is None: #feed zeros if we don't want to use inputs in the decoder
            dec_in = torch.zeros(x.shape).to(self.device)
        elif self.decoder_input == 'encoder_output':
            dec_in = out
        z, hiddens = self.decoder(dec_in, hiddens) #decode using the original input x and the learned representation of the sequence
        z = self.out(z) #final shared dense layer
        return z

##Attention-based

In [ ]:
class PositionalEncoding(nn.Module):
    '''positional encoding described in https://arxiv.org/abs/1706.03762'''
    def __init__(self, d_model, dropout, max_len):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len)
        w = 1.0 / (torch.pow(1000, torch.arange(0, d_model, 2) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(torch.einsum('i,j->ij', position, w))
        pe[:, 1::2] = torch.cos(torch.einsum('i,j->ij', position, w))
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x * math.sqrt(x.size(2))
        x = x + self.pe[:x.size(1)]
        return self.dropout(x)

In [ ]:
class WarmupScheduler(torch.optim.lr_scheduler._LRScheduler):
    '''warmup scheduler defined in https://arxiv.org/abs/1706.03762 note that this has to be called after each step, not each epoch'''
    def __init__(self, optimizer, warmup_steps = 400, d_model = 128, last_epoch=-1, verbose=False):
        self.warmup_steps = warmup_steps
        self.num_steps = 0
        self.d_model = d_model
        super(WarmupScheduler, self).__init__(optimizer, last_epoch, verbose)
        
    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.")

        self.num_steps += 1
        lr = 1 / math.sqrt(self.d_model) * min(1 / math.sqrt(self.num_steps), self.num_steps * (self.warmup_steps ** (-1.5)))
        return [lr] * len(self.optimizer.param_groups)

In [ ]:
class Transformer(GeneralModel):
    '''transformer model (the second model included in the report: we use lstm to create embeddings and append the input to the encoding before decoding)'''
    def __init__(self, input_size, output_size, max_len, embedding_size = 100, dropout_prob = 0.1, warmup_steps = 400, use_lstm_for_embeddings = False, append_input_decoding = True,
                 loss_name:str = 'mse', loss_lambda:float = 1.0, attention_heads = 1, num_encoder_blocks = 6, transformer_hidden_size = 2048, 
                 l2_penalty_lambda = 0.0, use_amsgrad=False, use_lr_sched = True, learning_rate = 1e-3):
        super().__init__(loss_name, learning_rate, use_lr_sched, 0.0, 0.0)
        
        self.seq_len = max_len
        self.warmup_steps = warmup_steps
        self.embedding_size = embedding_size
        self.num_features_transformer = embedding_size
        self.use_lstm = use_lstm_for_embeddings
        self.dropout = nn.Dropout(p=dropout_prob)
        if not self.use_lstm:
            self.embedding = nn.Linear(input_size, embedding_size, bias=False)
            self.pos_encoding = PositionalEncoding(self.num_features_transformer, dropout_prob, max_len)
        else:
            self.lstm = nn.LSTM(input_size=input_size, hidden_size=embedding_size, num_layers=1, dropout=0.0, batch_first=True)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.num_features_transformer, nhead=attention_heads, dropout=dropout_prob, activation='relu', dim_feedforward=transformer_hidden_size, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_blocks)

        self.decoder_size = self.num_features_transformer
        if append_input_decoding:
            self.decoder_size += self.embedding_size
        self.append_input_decoding = append_input_decoding

        self.out = nn.Linear(self.decoder_size, output_size)
        self.init_parameters()
        self.loss_lambda = loss_lambda

        self.automatic_optimization = False #we do training step by hand because we need to call the scheduler after each step, not epoch
        self.l2_penalty_lambda = l2_penalty_lambda
        self.use_amsgrad = use_amsgrad

    def forward(self, x): #used for inference
        if self.use_lstm:
            hiddens = (torch.zeros((1, x.shape[0], self.embedding_size)).to(self.device),
                    torch.zeros((1, x.shape[0], self.embedding_size)).to(self.device))
            x, _ = self.lstm(x, hiddens)
            x = self.dropout(F.selu(x))
        else:
            x = self.dropout(F.selu(self.embedding(x)))
            x = self.pos_encoding(x)
        
        x_enc = x #save encoding of x
        
        mask = self.get_causal_mask(self.seq_len)
        x = self.transformer_encoder(x, mask=mask)
        
        if self.append_input_decoding:
            x = torch.cat((x, x_enc), dim=-1) #concatenate embedding of x with its original encoding

        x = self.out(x)
        return x
    
    def get_causal_mask(self, seq_len): #mask to make the layers causal
        bmask = (torch.arange(seq_len).unsqueeze(0) <= torch.arange(seq_len).unsqueeze(1)).to(self.device)
        smask = torch.zeros((seq_len, seq_len)).to(self.device)
        smask[~bmask] = float('-inf')
        return smask

    def init_parameters(self): #init parameters with xavier
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.initial_learning_rate, betas=(0.9, 0.98), eps=1e-9, weight_decay=self.l2_penalty_lambda, amsgrad=self.use_amsgrad)
        if self.use_lr_sched:
            lr_sched = WarmupScheduler(optimizer, warmup_steps = self.warmup_steps, d_model = self.num_features_transformer)
            return {'optimizer': optimizer, 'lr_scheduler': lr_sched}
        return optimizer
    
    def training_step(self, batch, batch_idx):
        opt = self.optimizers()
        opt.zero_grad()
        loss = self.step(batch)
        self.manual_backward(loss)
        opt.step()
        sch = self.lr_schedulers()
        if sch is not None:
            sch.step()
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        if sch is not None:
            self.log('learning_rate', sch.get_last_lr()[0], on_step=True, prog_bar=True, logger=True)

##EMA

In [ ]:
class ExponentialMovingAverage():
    '''exponential moving average model (just used for some quick test) '''
    def __init__(self, alpha:float = 0.8):
        assert (0 <= alpha <= 1)
        self.alpha = alpha
        self.coeff = None
    
    def forward(self, x): #expect x.shape = [batch_size, seq_len, num_features]
        if self.coeff is None:
            seq_len = x.shape[1]
            num_features = x.shape[2] #also number of outputs
            self.coeff = torch.full((seq_len,num_features), 1-self.alpha)
            exps = torch.unsqueeze(torch.arange(start=seq_len-1, end=-1, step=-1), dim=1)
            self.coeff = self.coeff ** exps
            self.coeff = self.coeff * self.alpha
        return torch.sum(x * self.coeff, dim=1)
    
    def getFullPredictions(self, test_loader:DataLoader):
        y_pred = torch.Tensor()
        for batch in test_loader:
            y_pred = torch.cat((y_pred, self.forward(batch['x'])))
        return y_pred

    def searchBestAlpha(self, alphasToTry, loaderVal:DataLoader, target):
        bestAlpha = alphasToTry[0]
        bestError = -1
        for a in alphasToTry:
            self.alpha = a
            self.coeff = None
            y_pred = self.getFullPredictions(loaderVal)
            error = F.mse_loss(target, y_pred).item()
            if bestError < 0 or error < bestError:
                bestAlpha = a
                bestError = error
        self.alpha = bestAlpha
        return self.alpha
    
    def number_of_parameters(self):
        return 1

#Trading Strategies

In [ ]:
def nextday_trading_strategy(y_real, y_pred, price_idx = 0, threshold = 0):
    '''trading strategy described in the report'''
    if len(list(y_real.shape)) == 2:
        y_real = y_real[:, price_idx]
    if len(list(y_pred.shape)) == 2:
        y_pred = y_pred[:, price_idx]
    
    assert (y_real.shape == y_pred.shape)
    gain_list = []
    for i in range(len(y_real) - 1):
        prev = 0 if len(gain_list) == 0 else gain_list[-1]
        if y_pred[i+1] - y_pred[i] > threshold: #buy at i, sell at i+1
            gain_list += [prev + y_real[i+1] - y_real[i]]
        else:
            gain_list += [prev]
    return np.array(gain_list)

#Utilities

In [ ]:
def plotPrediction(y, y_pred, label_name, plot_name, log=True, idx_target=0, real_label='real', predicted_label='predicted'):
    '''plots the prediction of a model'''
    try:
        y = y.numpy()
        y_pred = y_pred.numpy()
    except:
        pass

    plt.clf()
    fig, ax = plt.subplots(figsize=(16, 9))

    shape = list(y.shape)
    if len(shape) == 2:
        y = y[:, idx_target]
        y_pred = y_pred[:, idx_target]

    x = np.arange(y.shape[0])
    ax.plot(x, y, label=real_label)
    ax.plot(x, y_pred, label=predicted_label)
    plt.title(plot_name)
    plt.xlabel('time')
    plt.ylabel(label_name)
    plt.legend()
    plt.show()
    if log:
        wandb.log({plot_name: ax})

In [ ]:
def getDataset(name:str, path:str, set_type:str, val_size:int, test_size:int, window_size:int = 7, cols_to_use = ['Open'], col_to_sort:str = 'Date', scaler = None, scale_sections:int = 5, scaler_type:str = 'minmax'):
    if name == 'AutoregressiveDataset':
        return AutoregressiveDataset(path, set_type, val_size, test_size, window_size, cols_to_use, col_to_sort, scaler, scale_sections, scaler_type)
    if name == 'AutoregressiveDistanceDataset':
        return AutoregressiveDistanceDataset(path, set_type, val_size, test_size, window_size, cols_to_use, col_to_sort, scaler, scale_sections, scaler_type)

def getModel(dataset_train, model_type:str, model_params:dict):
    if model_type == 'StackedLSTM':
        return StackedLSTM(input_size = dataset_train.num_features(), output_size = dataset_train.num_features(), **model_params)
    if model_type == 'Transformer':
        return Transformer(input_size = dataset_train.num_features(), output_size = dataset_train.num_features(), max_len = dataset_train.window_size, **model_params)
    if model_type == 'EncoderDecoderLSTM':
        return EncoderDecoderLSTM(input_size = dataset_train.num_features(), output_size = dataset_train.num_features(), **model_params)
    if model_type == 'MLP':
        return MLP(input_size = dataset_train.num_features(), output_size = dataset_train.num_features(), **model_params)

def plotConfusionMatrix(confusion_matrix, plot_name=None):
    '''
    plots the confusion matrix
    '''
    cm = confusion_matrix / np.array([np.sum(confusion_matrix, axis=1)]).T #normalize rows
    text = np.array([["count:{0}\n{1:.2f}%".format(confusion_matrix[i,j], cm[i,j]*100) for j in range(2)] for i in range(2)])
    plt.clf()
    fig, ax = plt.subplots(figsize=(5, 5))
    sns.heatmap(cm, annot=text, fmt='', ax=ax, cmap='viridis', square=True, annot_kws={"size":12})
    plt.ylabel("true label") 
    plt.xlabel("predicted label")
    if plot_name is not None: 
        plt.title(plot_name)
    plt.show()

def accuracy_fromcm(confusion_matrix):
    return (confusion_matrix[1,1] + confusion_matrix[0,0]) / np.sum(confusion_matrix)

def precision_fromcm(confusion_matrix):
    return confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[0,1])  #tp / (tp + fp)

def recall_fromcm(confusion_matrix):
    return confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[1,0]) #tp / (tp + fn)

def f1score_fromcm(confusion_matrix):
    p = precision_fromcm(confusion_matrix)
    r = recall_fromcm(confusion_matrix)
    return 2 * p * r / (p + r)

def mean_absolute_percentage_error(y, y_hat, eps=1e-8):
    return np.mean(np.abs((y - y_hat) / (y + eps)))

def confusion_matrix_from_predictions(y_true, y_pred, target_idx = 0, threshold = 0):
    '''uses the prediction to build a confusion matrix, where we predict 1 if the model predict to buy and 0 otherwise'''
    y_true_class = torch.zeros(y_true.shape[0] - 1)
    y_true_class[y_true[1:,target_idx] - y_true[:-1,target_idx] > threshold] = 1

    y_pred_class = torch.zeros(y_pred.shape[0] - 1)
    y_pred_class[y_pred[1:,target_idx] - y_pred[:-1,target_idx] > threshold] = 1

    tp = torch.sum(torch.logical_and(y_pred_class == y_true_class, y_true_class == 1).long())
    tn = torch.sum(torch.logical_and(y_pred_class == y_true_class, y_true_class == 0).long())
    fp = torch.sum(torch.logical_and(y_pred_class != y_true_class, y_true_class == 0).long())
    fn = torch.sum(torch.logical_and(y_pred_class != y_true_class, y_true_class == 1).long())
    return np.array([[tn, fp], [fn, tp]])

def computeMetric(y_test_true, y_test_hat, y_val_true, y_val_hat, y_train_true, y_train_hat, metric, metric_name, debug, log):
    '''computes the metric on test/val/train set and log the results'''
    test = metric(y_test_true, y_test_hat)
    val = metric(y_val_true, y_val_hat)
    train = metric(y_train_true, y_train_hat)
    if debug:
        print(f'{metric_name} (test):', test)
        print(f'{metric_name} (val):', val)
        print(f'{metric_name} (train):', train)
    if log:
        wandb.run.summary[f"{metric_name}_test"] = test
        wandb.run.summary[f"{metric_name}_val"] = val
        wandb.run.summary[f"{metric_name}_train"] = train

def performValidation(trainer, model, dataset_train, dataset_val, dataset_test, loader_test, debug, logging, plotsWithoutTeacher, nextday_trading_price):
    '''
    function to perform a full validation of the model
    '''
    trainer.test() #load best checkpoint
    trainer.test(dataloaders=loader_test)
    
    num_feat = dataset_train.get_targets().shape[1]

    if plotsWithoutTeacher:
        for i in range(num_feat):
            y_test, y_test_pred = dataset_test.plotPredictions(model, False, dataset_test.feature_name(i), f'test_predictions_{dataset_test.feature_name(i)}', logging, i)
            y_val, y_val_pred = dataset_val.plotPredictions(model, False, dataset_val.feature_name(i), f'val_predictions_{dataset_val.feature_name(i)}', logging, i)
            y_train, y_train_pred = dataset_train.plotPredictions(model, False, dataset_train.feature_name(i), f'train_predictions_{dataset_train.feature_name(i)}', logging, i)

    for i in range(num_feat):
        y_test, y_test_pred_teach = dataset_test.plotPredictions(model, True, dataset_test.feature_name(i), f'test_teacher_predictions_{dataset_test.feature_name(i)}', logging, i)
        y_val, y_val_pred_teach = dataset_val.plotPredictions(model, True, dataset_val.feature_name(i), f'val_teacher_predictions_{dataset_val.feature_name(i)}', logging, i)
        y_train, y_train_pred_teach = dataset_train.plotPredictions(model, True, dataset_train.feature_name(i), f'train_teacher_predictions_{dataset_train.feature_name(i)}', logging, i)

    test_prices = dataset_test.get_real_prices()
    test_pred_prices = dataset_test.get_predicted_prices(model, True)
    val_prices = dataset_val.get_real_prices()
    val_pred_prices = dataset_val.get_predicted_prices(model, True)
    train_prices = dataset_train.get_real_prices()
    train_pred_prices = dataset_train.get_predicted_prices(model, True)
    if dataset_train.is_prediction_different_from_price(): #the dataset predicts something different from the actual price: now we plot the actual price using the predictions
        prices = dataset_train.can_output_prices()
        for i in range(len(prices)):
            plotPrediction(test_prices, test_pred_prices, prices[i], f'test_teacher_prices_{prices[i]}', log=logging, idx_target=i)
            plotPrediction(val_prices, val_pred_prices, prices[i], f'val_teacher_prices_{prices[i]}', log=logging, idx_target=i)
            plotPrediction(train_prices, train_pred_prices, prices[i], f'train_teacher_prices_{prices[i]}', log=logging, idx_target=i)

    #use trading strategies
    if nextday_trading_price >= 0:
        gain_test = nextday_trading_strategy(test_prices, test_pred_prices, price_idx = nextday_trading_price)
        gain_test_optimal = nextday_trading_strategy(test_prices, test_prices, price_idx = nextday_trading_price)
        gain_val = nextday_trading_strategy(val_prices, val_pred_prices, price_idx = nextday_trading_price)
        gain_val_optimal = nextday_trading_strategy(val_prices, val_prices, price_idx = nextday_trading_price)
        gain_train = nextday_trading_strategy(train_prices, train_pred_prices, price_idx = nextday_trading_price)
        gain_train_optimal = nextday_trading_strategy(train_prices, train_prices, price_idx = nextday_trading_price)
        
        plotPrediction(gain_test_optimal, gain_test, 'next-day-gain', 'test next-day-gain', log=logging, real_label='optimal', predicted_label='using model')
        plotPrediction(gain_val_optimal, gain_val, 'next-day-gain', 'val next-day-gain', log=logging, real_label='optimal', predicted_label='using model')
        plotPrediction(gain_train_optimal, gain_train, 'next-day-gain', 'train next-day-gain', log=logging, real_label='optimal', predicted_label='using model')
    
    #log summaries
    
    computeMetric(y_test, y_test_pred_teach, y_val, y_val_pred_teach, y_train, y_train_pred_teach, r2_score, 'r2_score', debug, logging) #r2 score
    computeMetric(y_test, y_test_pred_teach, y_val, y_val_pred_teach, y_train, y_train_pred_teach, mean_absolute_error, 'MAE', debug, logging) #MAE
    computeMetric(y_test, y_test_pred_teach, y_val, y_val_pred_teach, y_train, y_train_pred_teach, lambda y, y_hat: np.sqrt(mean_squared_error(y, y_hat)), 'RMSE', debug, logging) #RMSE
    computeMetric(test_prices, test_pred_prices, val_prices, val_pred_prices, train_prices, train_pred_prices, r2_score, 'r2_score_on_prices', debug, logging) #r2 score on prices
    try:
        computeMetric(test_prices, test_pred_prices, val_prices, val_pred_prices, train_prices, train_pred_prices, mean_absolute_percentage_error, 'MAPE_on_prices', debug, logging) #MAPE on prices
    except:
        if debug:
            print('mape not computed because of negative values')

    if nextday_trading_price >= 0: #evaluate the model as if it was a classifier (of course it will perform worse than an actual classifier, but it's interesting to look at these)
        cm_test = confusion_matrix_from_predictions(y_test, y_test_pred_teach)
        cm_val = confusion_matrix_from_predictions(y_val, y_val_pred_teach)
        cm_train = confusion_matrix_from_predictions(y_train, y_train_pred_teach)
        if debug:
            plotConfusionMatrix(cm_test, plot_name='confusion matrix test')
            plotConfusionMatrix(cm_val, plot_name='confusion matrix val')
            plotConfusionMatrix(cm_train, plot_name='confusion matrix train')
        acc_test = accuracy_fromcm(cm_test)
        acc_val = accuracy_fromcm(cm_val)
        acc_train = accuracy_fromcm(cm_train)
        f1_test = f1score_fromcm(cm_test)
        f1_val = f1score_fromcm(cm_val)
        f1_train = f1score_fromcm(cm_train)

    if debug:
        if nextday_trading_price >= 0:
            print('gain next-day strategy (test):', gain_test[-1])
            print('gain next-day strategy (val):', gain_val[-1])
            print('gain next-day strategy (train):', gain_train[-1])
            print('optimal gain next-day strategy (test):', gain_test_optimal[-1])
            print('optimal gain next-day strategy (val):', gain_val_optimal[-1])
            print('optimal gain next-day strategy (train):', gain_train_optimal[-1])

            print('accuracy test:', acc_test)
            print('accuracy val:', acc_val)
            print('accuracy train:', acc_train)
            print('f1 test:', f1_test)
            print('f1 val:', f1_val)
            print('f1 train:', f1_train)
    if logging:
        if nextday_trading_price >= 0:
            wandb.run.summary['gain_next-day_strategy_test'] = gain_test[-1]
            wandb.run.summary['gain_next-day_strategy_val'] = gain_val[-1]
            wandb.run.summary['gain_next-day_strategy_train'] = gain_train[-1]
            wandb.run.summary['optimal_gain_next-day_strategy_test'] = gain_test_optimal[-1]
            wandb.run.summary['optimal_gain_next-day_strategy_val'] = gain_val_optimal[-1]
            wandb.run.summary['optimal_gain_next-day_strategy_train'] = gain_train_optimal[-1]

            wandb.run.summary["accuracy_test"] = acc_test
            wandb.run.summary["accuracy_val"] = acc_val
            wandb.run.summary["accuracy_train"] = acc_train
            wandb.run.summary["f1_test"] = f1_test
            wandb.run.summary["f1_val"] = f1_val
            wandb.run.summary["f1_train"] = f1_train
            wandb.run.summary["conf_matrix_test"] = cm_test
            wandb.run.summary["conf_matrix_val"] = cm_val
            wandb.run.summary["conf_matrix_train"] = cm_train


In [ ]:
def run_experiment(dataset_type:str, dataset_path:str, window_size:int, scale_sections:int, batch_size:int, max_epochs:int, model_type:str, model_params:dict, 
                   val_size:int = 365, test_size:int = 365, wandb_project_name:str = None, wandb_entity:str = None, debug:bool = True, logging:bool = True, 
                   nextday_trading_price=0, plotsWithoutTeacher:bool=False, gpus:int = 0, cols_to_use=['Open'], scaler_type='minmax', eval_every_n_epochs=10, early_stopping=False,
                   min_epochs=1, early_stopping_patience=10):
    '''
    function to run a complete experiment
    '''
    
    dataset_train = getDataset(dataset_type, dataset_path, 'train', val_size=val_size, test_size=test_size, window_size=window_size, scale_sections=scale_sections, cols_to_use=cols_to_use, scaler_type=scaler_type)
    dataset_val = getDataset(dataset_type, dataset_path, 'val', val_size=val_size, test_size=test_size, window_size=window_size, scaler=dataset_train.scaler, cols_to_use=cols_to_use, scaler_type=scaler_type)
    dataset_test = getDataset(dataset_type, dataset_path, 'test', val_size=val_size, test_size=test_size, window_size=window_size, scaler=dataset_train.scaler, cols_to_use=cols_to_use, scaler_type=scaler_type)

    loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False)
    loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
    loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

    if debug:
        print('train set size:', len(dataset_train))
        print('validation set size:', len(dataset_val))
        print('test set size:', len(dataset_test))
    
    logger = None
    if logging:
        wandb.init(project=wandb_project_name, entity=wandb_entity)
        dataset_train.log(wandb.config)
        wandb.config.batch_size = batch_size
        wandb.config.max_epochs = max_epochs
        wandb.config.model_type = model_type
        wandb.config.model_params = model_params
        logger = WandbLogger(project=WANDB_PROJECT_NAME, log_model=True)
    
    model = getModel(dataset_train, model_type, model_params)

    early_stop_callback = EarlyStopping(monitor='val_loss_epoch', min_delta=0.00, patience=early_stopping_patience, verbose=False, strict=False)
    callbacks = []
    if early_stopping:
        callbacks = [early_stop_callback]
    trainer = pl.Trainer(gpus=gpus, max_epochs = max_epochs, logger=logger, log_every_n_steps=30, 
                         check_val_every_n_epoch=eval_every_n_epochs, callbacks=callbacks, min_epochs=min_epochs) #, deterministic=True
    if logger is not None:
        logger.watch(model)
    trainer.fit(model, loader_train, loader_val)

    #validation
    performValidation(trainer, model, dataset_train, dataset_val, dataset_test, loader_test, debug, logging, plotsWithoutTeacher, nextday_trading_price)

    if logging:
        wandb.finish()

#Experiments

In [ ]:
WANDB_PROJECT_NAME = 'StockPredictions'
SEED = 42

torch.set_default_dtype(torch.float64)
pl.seed_everything(SEED)

dsPath = '/content/drive/My Drive/DL/BTC-USD.csv'
#dsPath = '/content/drive/My Drive/DL/GOOG.csv'
#dsPath = '/content/drive/My Drive/DL/AAPL.csv'

##MLP

In [ ]:
#============= MLP
run_experiment('AutoregressiveDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 400, model_type = 'MLP', model_params = {'hidden_size':200, 'loss_name':'mse', 
                'learning_rate':0.001, 'use_lr_sched':True, 'lr_decay_step': 150, 'lr_decay_gamma': 0.1}, 
                val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, cols_to_use=['Open'], logging=True)

run_experiment('AutoregressiveDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 400, model_type = 'MLP', model_params = {'hidden_size':200, 'loss_name':'mse+exp_trend4', 
                'learning_rate':0.001, 'use_lr_sched':True, 'lr_decay_step': 150, 'lr_decay_gamma': 0.1}, 
                val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, cols_to_use=['Open'], logging=True)

run_experiment('AutoregressiveDistanceDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 400, model_type = 'MLP', model_params = {'hidden_size':200, 'loss_name':'mse', 
                'learning_rate':0.001, 'use_lr_sched':True, 'lr_decay_step': 150, 'lr_decay_gamma': 0.1}, 
                val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, cols_to_use=['Open'], logging=True)

##LSTM

In [ ]:
#============= LSTM
for i in range(3):
    run_experiment('AutoregressiveDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 300, model_type = 'StackedLSTM', model_params = {'hidden_size':100, 'num_layers':1, 
                    'feed_forward_depth':1, 'learning_rate':1e-3, 'use_lr_sched':True, 'lr_decay_step': 125, 'lr_decay_gamma': 0.1, 'loss_name':'mse', 'loss_lambda':0.1}, 
                    cols_to_use=['Open'], scaler_type='minmax', val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, logging=True)
    
    run_experiment('AutoregressiveDistanceDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 300, model_type = 'StackedLSTM', model_params = {'hidden_size':100, 'num_layers':1, 
                    'feed_forward_depth':1, 'learning_rate':1e-3, 'use_lr_sched':True, 'lr_decay_step': 125, 'lr_decay_gamma': 0.1, 'loss_name':'mse', 'loss_lambda':0.1}, 
                    cols_to_use=['Open'], scaler_type='minmax', val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, logging=True)
    
    run_experiment('AutoregressiveDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 300, model_type = 'StackedLSTM', model_params = {'hidden_size':100, 'num_layers':1, 
                    'feed_forward_depth':1, 'learning_rate':1e-3, 'use_lr_sched':True, 'lr_decay_step': 125, 'lr_decay_gamma': 0.1, 'loss_name':'mse+exp_trend4', 'loss_lambda': 0.1}, 
                    cols_to_use=['Open'], scaler_type='minmax', val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, logging=True)

##Attention

In [ ]:
for i in range(3):
    run_experiment('AutoregressiveDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 200, model_type = 'Transformer', model_params = {'embedding_size':128, 
                'dropout_prob':0.1, 'loss_name':'mse+exp_trend4', 'attention_heads': 2, 'num_encoder_blocks': 2, 'transformer_hidden_size': 2048, 
                'warmup_steps': 200, 'loss_lambda':1, 'use_lstm_for_embeddings':True, 'append_input_decoding': True, 'l2_penalty_lambda':0, 'use_amsgrad': False, 'use_lr_sched':True}, 
                cols_to_use=['Open'], scaler_type='minmax', val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, logging=True,
                eval_every_n_epochs=1, early_stopping=False, min_epochs=150, early_stopping_patience=10)
        
    run_experiment('AutoregressiveDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 200, model_type = 'Transformer', model_params = {'embedding_size':128, 
                'dropout_prob':0.1, 'loss_name':'mse', 'attention_heads': 2, 'num_encoder_blocks': 2, 'transformer_hidden_size': 2048, 
                'warmup_steps': 200, 'loss_lambda':1, 'use_lstm_for_embeddings':True, 'append_input_decoding': True, 'l2_penalty_lambda':0, 'use_amsgrad': False, 'use_lr_sched':True}, 
                cols_to_use=['Open'], scaler_type='minmax', val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, logging=True,
                eval_every_n_epochs=1, early_stopping=False, min_epochs=150, early_stopping_patience=10)

    run_experiment('AutoregressiveDistanceDataset', dsPath, window_size = 25, scale_sections = 1, batch_size = 64, max_epochs = 100, model_type = 'Transformer', model_params = {'embedding_size':128, 
                'dropout_prob':0.1, 'loss_name':'mse', 'attention_heads': 2, 'num_encoder_blocks': 2, 'transformer_hidden_size': 2048, 
                'warmup_steps': 200, 'loss_lambda':1, 'use_lstm_for_embeddings':True, 'append_input_decoding': True, 'l2_penalty_lambda':0, 'use_amsgrad': False, 'use_lr_sched':True}, 
                cols_to_use=['Open'], scaler_type='minmax', val_size = 365, test_size = 365, wandb_project_name = WANDB_PROJECT_NAME, wandb_entity = 'mirko222', gpus=1, logging=True,
                eval_every_n_epochs=1, early_stopping=False, min_epochs=1, early_stopping_patience=10)